# Imports

In [1]:
import aurorax
import datetime
import pandas as pd
import pprint

# Search

In [4]:
# set values
start_dt = datetime.datetime(2019, 1, 1, 0, 0, 0)
end_dt = datetime.datetime(2019, 1, 1, 23, 59, 59)
programs = ["themis-asi"]
platforms = ["gillam"]
instrument_types = ["panchromatic ASI"]
data_product_types = ["keogram"]

In [5]:
# perform search
s = aurorax.data_products.search(start_dt,
                                 end_dt,
                                 programs=programs,
                                 platforms=platforms,
                                 instrument_types=instrument_types,
                                 data_product_types=data_product_types,
                                 show_progress=True)

[2020-11-23 00:16:12.314559] Search object created
[2020-11-23 00:16:12.356826] Request submitted
[2020-11-23 00:16:12.357169] Request ID: 4c22fd63-3a11-4392-a80e-16ef6c5177bb
[2020-11-23 00:16:12.357372] Request details available at: http://api.staging.aurorax.space/api/v1/data_products/requests/4c22fd63-3a11-4392-a80e-16ef6c5177bb
[2020-11-23 00:16:12.407903] Checking for data ...
[2020-11-23 00:16:12.458478] Request has data available, retrieving it ...
[2020-11-23 00:16:12.501985] Retrieved 5 Bytes of data containing 0 records, completed search request


In [ ]:
# output data as a pandas dataframe
df = pd.DataFrame(s["data"])
df.sort_values("epoch")  # sort by the timestamp

# Do the search asynchonously

Under the hood a data products search is asynchronous. This adds some more complexity for the user, but also opens the search up to some additional possibilities. The main advantage is being able to perform queries for a large timeframe. These queries will take a bit longer and could exceed several minutes and more than a gigabyte of data. Separating the request and the resulting data into two different API calls allows flexibility in duration and size of the overall request.

In [7]:
# set up the request
s = aurorax.data_products.Search(start_dt,
                                 end_dt,
                                 programs=programs,
                                 platforms=platforms,
                                 instrument_types=instrument_types,
                                 data_product_types=data_product_types)
print(s)

{'data': [],
 'data_available': False,
 'data_product_types': ['keogram'],
 'data_url': '',
 'end_dt': datetime.datetime(2019, 1, 1, 23, 59, 59),
 'executed': False,
 'instrument_types': ['panchromatic ASI'],
 'logs': [],
 'metadata_filters': [],
 'platforms': ['gillam'],
 'programs': ['themis-asi'],
 'query': {},
 'request': None,
 'request_id': '',
 'request_url': '',
 'start_dt': datetime.datetime(2019, 1, 1, 0, 0),
 'status': {}}


In [8]:
# execute the search
s.execute()
print(s)

{'data': [],
 'data_available': False,
 'data_product_types': ['keogram'],
 'data_url': '',
 'end_dt': datetime.datetime(2019, 1, 1, 23, 59, 59),
 'executed': True,
 'instrument_types': ['panchromatic ASI'],
 'logs': [],
 'metadata_filters': [],
 'platforms': ['gillam'],
 'programs': ['themis-asi'],
 'query': {'data_product_type_filters': ['keogram'],
           'data_sources': {'data_product_metadata_filters': [],
                            'instrument_types': ['panchromatic ASI'],
                            'platforms': ['gillam'],
                            'programs': ['themis-asi']},
           'end': '2019-01-01T23:59:59',
           'start': '2019-01-01T00:00:00'},
 'request': <AuroraXResponse [202]>,
 'request_id': 'c05d0f64-7d7d-499c-bfb4-539765889c2e',
 'request_url': 'http://api.staging.aurorax.space/api/v1/data_products/requests/c05d0f64-7d7d-499c-bfb4-539765889c2e',
 'start_dt': datetime.datetime(2019, 1, 1, 0, 0),
 'status': {}}


In [10]:
# get request status
status = aurorax.data_products.get_request_status(s.request_id)
pprint.pprint(status)

{'data': {'logs': [{'level': 'debug',
                    'summary': 'Search request arrived',
                    'timestamp': '2020-11-23T00:17:24.116308Z'},
                   {'level': 'info',
                    'summary': 'Starting query for '
                               'c05d0f64-7d7d-499c-bfb4-539765889c2e',
                    'timestamp': '2020-11-23T00:17:24.118694Z'},
                   {'level': 'info',
                    'summary': 'Finished query in 0.00 seconds.',
                    'timestamp': '2020-11-23T00:17:24.121918Z'},
                   {'level': 'debug',
                    'summary': 'Starting to write data to file',
                    'timestamp': '2020-11-23T00:17:24.124773Z'},
                   {'level': 'info',
                    'summary': 'Finished writing data, found 0 records in 0.00 '
                               'seconds.',
                    'timestamp': '2020-11-23T00:17:24.126991Z'},
                   {'level': 'debug',
              

In [11]:
# just retrieve the logs for the request
logs = aurorax.data_products.get_request_logs(s.request_id)
pprint.pprint(logs)

{'data': [{'level': 'debug',
           'summary': 'Search request arrived',
           'timestamp': '2020-11-23T00:17:24.116308Z'},
          {'level': 'info',
           'summary': 'Starting query for c05d0f64-7d7d-499c-bfb4-539765889c2e',
           'timestamp': '2020-11-23T00:17:24.118694Z'},
          {'level': 'info',
           'summary': 'Finished query in 0.00 seconds.',
           'timestamp': '2020-11-23T00:17:24.121918Z'},
          {'level': 'debug',
           'summary': 'Starting to write data to file',
           'timestamp': '2020-11-23T00:17:24.124773Z'},
          {'level': 'info',
           'summary': 'Finished writing data, found 0 records in 0.00 seconds.',
           'timestamp': '2020-11-23T00:17:24.126991Z'},
          {'level': 'debug',
           'summary': 'File size is 5 bytes.',
           'timestamp': '2020-11-23T00:17:24.128741Z'},
          {'level': 'info',
           'summary': 'Finished search in 0.01 seconds and found 0 records.',
           'times

In [12]:
# wait for the data
s.wait()
s.update_status()

In [ ]:
# get data
s.get_data()

# show data as pandas dataframe
df = pd.DataFrame(s.data)
df.sort_values("epoch")  # sort by the timestamp